In [2]:
import arxiv
import requests
from pathlib import Path
## 요약 및 리뷰하려는 논문을 아카이브에서 id를 가져옵니다. 
## 이후 다운을 받습니다. 
def download_arxiv_pdf(pdf_url, save_path):
    response = requests.get(pdf_url, stream=True)
    if response.status_code == 200:
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)

arxiv_id = "2406.11813"
search = arxiv.Search(id_list=[arxiv_id])
for result in search.results():
    break
save_path = Path('references') / f"0-{result.title[:15]}.pdf"
pdf_url = result.pdf_url
download_arxiv_pdf(pdf_url, save_path)

print(f"📌 Title: {result.title}")
print(f"📝 Authors: {', '.join([author.name for author in result.authors])}")
print(f"📅 Submitted: {result.published}")
print(f"🔗 PDF Link: {result.pdf_url}")
print(f"📝 Abstract:\n{result.summary}")

/tmp/ipykernel_950/3175900557.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


📌 Title: How Do Large Language Models Acquire Factual Knowledge During Pretraining?
📝 Authors: Hoyeon Chang, Jinho Park, Seonghyeon Ye, Sohee Yang, Youngkyung Seo, Du-Seong Chang, Minjoon Seo
📅 Submitted: 2024-06-17 17:54:40+00:00
🔗 PDF Link: http://arxiv.org/pdf/2406.11813v3
📝 Abstract:
Despite the recent observation that large language models (LLMs) can store
substantial factual knowledge, there is a limited understanding of the
mechanisms of how they acquire factual knowledge through pretraining. This work
addresses this gap by studying how LLMs acquire factual knowledge during
pretraining. The findings reveal several important insights into the dynamics
of factual knowledge acquisition during pretraining. First, counterintuitively,
we observe that pretraining on more data shows no significant improvement in
the model's capability to acquire and maintain factual knowledge. Next, there
is a power-law relationship between training steps and forgetting of
memorization and generalizatio

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader
# 여기는 손수 정리해줘야 합니다.
# 논문을 section 별로 나누는 것입니다.

loader = UnstructuredPDFLoader(save_path)
documents = loader.load()
output = {
    "Title":result.title,
    "Authors":'\n'.join([author.name for author in result.authors]),
    "Submitted":str(result.published),
    "Abstract":result.summary,
    # 아래는 논문의 reference 인용 횟수를 count하기 위한 기초 작업입니다.
    "Introduction":documents[0].page_content.split("Introduction\n\n")[-1].split("Related Work\n\n")[0],
    "Related Work":documents[0].page_content.split("Related Work\n\n")[-1].split("Experimental Setup\n\n")[0],
    "Experimental Setup":documents[0].page_content.split("Experimental Setup\n\n")[-1].split("Results\n\n")[0],
    "Results":documents[0].page_content.split("Results\n\n")[-1].split("Discussion and Conclusions\n\n")[0],
    "Discussion and Conclusions":documents[0].page_content.split("Discussion and Conclusions\n\n")[-1].split("References\n\n")[0],
    "References":documents[0].page_content.split("References\n\n")[-1].split("Appendix\n\n")[0],
    "Appendix":documents[0].page_content.split("Appendix\n\n")[-1],
}
reference_count_keys = ["Introduction", "Related Work", "Experimental Setup", "Results", "Discussion and Conclusions"]
content_keys = ["Title","Authors","Submitted","Abstract","Introduction", "Related Work", "Experimental Setup", "Results", "Discussion and Conclusions"]

In [4]:
# "\n" 사이의 문자가 threshold 미만이면 제거합니다.
threshold = 25

for key in reference_count_keys:
    result = []
    for text in output[key].split("\n"):
        if len(text) >= threshold:
            result.append(text)
    output[key] = "\n".join(result)

In [5]:
from openai import OpenAI
import openai
from dotenv import load_dotenv
import os
import re
import json

# .env가 아닌 config.env에서 환경 변수 로드
load_dotenv(dotenv_path="config.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

def message_to_openai(user, model = 'gpt-4o-mini'):

    response = client.chat.completions.create(
        model=model,
        store=True,
        messages=[{"role": "user", "content": user}],
        temperature=0
    )
    return response

user_prompt_template = """
논문: {essay}

다음 논문의 내용을 요약해주세요. 항목은 다음과 같습니다.

## 항목 정보
1. 기본 정보: 논문의 제목과 저자, 모두 영문으로 작성합니다. 
2. 연구 목적: 이 논문이 가진 문제의식과 이에 대한 설명을 작성합니다. 이때 문제의식은 명사형으로 끝내고 설명은 300자에서 500자로 서술합니다. 
3. 연구 방법: 해당 논문이 연구한 실험 방법과 데이터, 모델, 분석 방법 등을 작성합니다. 연구 방법은 가능한 한 구체적으로 기술합니다.
4. 주요 결과: 논문의 연구 성과를 요약합니다. 이때 연구의 핵심적인 발견과 논문의 주요 기여를 강조합니다.
5. 결론 및 시사점: 논문의 결론과 연구의 의미를 정리합니다. 또한 연구의 한계점과 향후 연구 방향을 포함하여 설명합니다.

## 응답 예시

### 1. 기본 정보
1) 제목: <<논문 제목(영문)>>
2) 저자: <<논문 저자(영문)>>

### 2. 연구 목적
1) 문제의식: <<문제 의식을 명사형으로 구성>>
2) 설명: <<문제 의식에 대한 설명을 300자에서 500자로 서술>>

### 3. 연구 방법
1) 실험 방법: <<연구에서 사용한 실험 방법을 기술>>
2) 데이터: <<사용된 데이터의 출처 및 특성을 설명>>
3) 모델 및 분석 방법: <<적용된 모델과 분석 기법을 설명>>

### 4. 주요 결과
1) 연구의 주요 발견: <<논문의 핵심 결과를 요약>>
2) 기여 및 성과: <<연구가 기존 연구 대비 기여한 점을 설명>>

### 5. 결론 및 시사점
1) 결론: <<연구의 주요 결론을 요약>>
2) 시사점: <<연구가 시사하는 바와 활용 가능성을 설명>>
3) 연구의 한계: <<논문에서 언급한 연구의 한계를 기술>>
4) 향후 연구 방향: <<추가 연구가 필요한 부분을 설명>>

가능한 한 논문의 핵심 내용을 유지하면서 간결하고 명확하게 요약해주세요.
"""

In [6]:
essay = "\n\n".join([output[key] for key in content_keys])
user_message = user_prompt_template.format(essay=essay)
response = message_to_openai(user_message)
output['Basic_summary'] = response.choices[0].message.content

In [7]:
print(output['Basic_summary'])

### 1. 기본 정보
1) 제목: How Do Large Language Models Acquire Factual Knowledge During Pretraining?
2) 저자: Hoyeon Chang, Jinho Park, Seonghyeon Ye, Sohee Yang, Youngkyung Seo, Du-Seong Chang, Minjoon Seo

### 2. 연구 목적
1) 문제의식: 대형 언어 모델의 사실적 지식 습득 메커니즘
2) 설명: 최근 대형 언어 모델(LLM)이 상당한 사실적 지식을 저장할 수 있다는 관찰에도 불구하고, 이들이 사전 훈련 중 사실적 지식을 어떻게 습득하는지에 대한 이해는 제한적이다. 본 연구는 LLM의 사실적 지식 습득 과정을 분석하여, 훈련 데이터의 양, 훈련 단계, 모델 크기, 배치 크기 등의 다양한 조건이 지식 습득에 미치는 영향을 조사한다. 이를 통해 LLM의 훈련 동역학을 이해하고, 지식 습득의 메커니즘을 규명하고자 한다.

### 3. 연구 방법
1) 실험 방법: 연구진은 LLM의 중간 사전 훈련 체크포인트를 사용하여, 이전에 접하지 않은 목표 지식을 주입하고, 다양한 조건에서 사실적 지식의 습득 과정을 모니터링하였다. 
2) 데이터: FICTIONAL KNOWLEDGE 데이터셋을 구성하여, 허구적이지만 현실적인 개체에 대한 설명을 포함한 문장을 주입하였다. 이 데이터셋은 LLM이 훈련 중에 접하지 않은 지식을 포함한다.
3) 모델 및 분석 방법: OLMo 모델을 사용하여, 주입된 지식에 대한 로그 확률을 평가하고, 지식 습득의 효과성 및 유지 가능성을 측정하기 위해 다양한 메트릭을 정의하였다.

### 4. 주요 결과
1) 연구의 주요 발견: LLM은 사실적 지식을 습득할 때, 미세한 확률의 누적을 통해 이루어지며, 훈련 중 지식이 주어지지 않으면 잊어버리는 경향이 있다. 또한, 모델 크기가 클수록 지식 습득의 효과성이 높아지지만, 훈련 데이터의 양이 많아져도 효과성은 크게 개선되지 않는다.
2) 기여 및 성과:

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 섹션 별로 2000글자 단위로 쪼갭니다.
# 쪼개는 목적은 인용 부를 확인하기 위함입니다.
def preprocess(text, title, key, chunk_size=2000, chunk_overlap=0):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap  
    )

    documents = text_splitter.create_documents([text])

    # 각 문서에 메타데이터 추가
    for doc in documents:
        doc.metadata = {"Title": title, "Key": key} 

    return documents

for key in reference_count_keys:
    output[key] = preprocess(output[key], output['Title'], key)

In [12]:
user_message_template = """
다음은 논문에서 reference 부분만 발췌한 것입니다. 
이를 보고 논문의 제목과 저자를 출력 형식에 따라 JSON 형태로 정리해주세요.

References:
{references}

출력 형식:
```json
{{
1 : {{"Title":<<논문의 제목1>>, "Authors":<<논문의 저자1>>}},
2 : {{"Title":<<논문의 제목2>>, "Authors":<<논문의 저자2>>}},
3 : {{"Title":<<논문의 제목3>>, "Authors":<<논문의 저자3>>}}
}}
```
"""

In [13]:
from copy import deepcopy
user_message = user_message_template.format(references=output['References'])
response = message_to_openai(user_message)

text = response.choices[0].message.content
text = re.sub("```json","",text)
text = re.sub("```","",text)
json_data = json.loads(text)

reference_dict = {}
for key, dict_data in json_data.items():
    dict_data['Counter'] = 0
    dict_data['Context'] = []
    reference_dict[key] = dict_data

output['References'] = deepcopy(reference_dict)

In [15]:
user_message_template_dict = {
    '0': """
다음은 논문의 일부와 참고문헌 목록을 정리한 것입니다. 
논문 본문에서 참고문헌이 인용되었다면, 해당 참고문헌의 "Counter"에 인용 횟수를 기록하고,  
"Context"에는 해당 참고문헌이 인용된 부분을 **300자 이내**로 발췌해서 저장하세요.  
**인용되지 않은 참고문헌은 출력하지 말고, 인용된 참고문헌만 JSON 형식으로 출력하세요.**

**조건**
- 인용 표시: 논문 본문에서 참고문헌이 **[숫자]** 형식 (`[5]`, `[27]` 등)으로 인용된 경우만 추출하세요.
- 참고문헌이 여러 번 인용되었을 경우, 각 인용 부분을 `"Context"` 배열에 저장하세요.
- `"Context"`에 포함되는 인용된 문장은 **최대 300자 이내**로 유지하세요.

**출력 형식**
```json
{{
    "27": {{
        "Title": "참고문헌 제목1",
        "Counter": 3,
        "Context": ["인용된 부분1", "인용된 부분2", "인용된 부분3"]
    }},
    "5": {{
        "Title": "참고문헌 제목2",
        "Counter": 1,
        "Context": ["인용된 부분1"]
    }}
}}
```

**References**
{references}

**Essay**
{essay}

""",
    "1" : """
논문의 일부와 참고문헌 목록이 제공되었습니다. 
논문 본문에서 특정 참고문헌이 인용되었다면, 해당 참고문헌의 **인용 횟수**를 `"Counter"`에 기록하고, 
**인용된 문장**을 `"Context"`에 최대 300자 이내로 발췌하여 저장하세요.  
**한 번도 인용되지 않은 참고문헌은 출력에서 제외됩니다.**

## 조건
- 인용표시: 본문에서 참고문헌이 **[숫자]** (`[10]`, `[3]` 등) 형식으로 인용된 경우만 포함합니다.
- 동일한 참고문헌이 여러 번 인용되면 `"Context"` 배열에 모든 인용 부분을 포함합니다.
- `"Context"`의 개별 항목은 **최대 300자 이하**를 유지해야 합니다.

## 출력 예시
```json
{{
    "10": {{
        "Title": "참고문헌 제목1",
        "Counter": 2,
        "Context": ["첫 번째 인용 문장", "두 번째 인용 문장"]
    }},
    "3": {{
        "Title": "참고문헌 제목2",
        "Counter": 1,
        "Context": ["인용된 문장"]
    }}
}}
```

## References
{references}

## Essay
{essay}
""",

    "2": """
논문의 본문과 참고문헌 목록을 정리해야 합니다.  
논문에서 특정 참고문헌이 인용되었을 경우,
해당 참고문헌의 **인용 횟수(`"Counter"`)**와 **인용 문장(`"Context"`)**을 JSON으로 출력하세요.  
**본문에서 한 번도 인용되지 않은 참고문헌은 출력하지 않습니다.**

## 조건
- 인용 표시: 참고문헌이 본문에서 **[숫자]** (`[7]`, `[19]` 등) 형식으로 등장하는 경우만 포함합니다.
- 동일한 참고문헌이 여러 번 인용되었을 경우, `"Context"` 배열에 각 인용 문장을 저장합니다.
- `"Context"`의 각 문장은 **300자 이내**로 제한해야 합니다.

## 출력 예시
```json
{{
    "7": {{
        "Title": "참고문헌 제목1",
        "Counter": 3,
        "Context": ["첫 번째 인용 문장", "두 번째 인용 문장", "세 번째 인용 문장"]
    }},
    "19": {{
        "Title": "참고문헌 제목2",
        "Counter": 1,
        "Context": ["인용된 문장"]
    }}
}}
```

## References
{references}

## Essay
{essay}
""",

    "3": """
논문의 본문과 참고문헌 목록이 주어졌습니다.  
논문에서 참고문헌이 인용되었다면,
해당 참고문헌의 인용 횟수를 `"Counter"`에 기록하고,  
인용된 문장을 300자 이내로 정리하여 `"Context"` 배열에 저장하세요.  
**인용되지 않은 참고문헌은 출력하지 않습니다.**

## 조건
- 인용 표시: 참고문헌이 본문에서 **[숫자]** (`[6]`, `[22]` 등) 형식으로 등장한 경우만 포함합니다.
- 동일한 참고문헌이 여러 번 인용된 경우 `"Context"` 배열에 모든 인용 문장을 기록합니다.
- `"Context"`의 개별 항목은 **최대 300자 이하**로 제한합니다.

## 출력 예시
```json
{{
    "6": {{
        "Title": "참고문헌 제목1",
        "Counter": 4,
        "Context": ["첫 번째 인용 문장", "두 번째 인용 문장", "세 번째 인용 문장", "네 번째 인용 문장"]
    }},
    "22": {{
        "Title": "참고문헌 제목2",
        "Counter": 2,
        "Context": ["첫 번째 인용 문장", "두 번째 인용 문장"]
    }}
}}
```

## References
{references}

## Essay
{essay}
""",

    "4": """
논문 본문에서 참고문헌이 인용된 경우, 해당 참고문헌의 **인용 횟수**와 **인용 문장**을 정리하여 JSON 형식으로 출력하세요.  
**인용되지 않은 참고문헌은 제외합니다.**

## 조건
- 인용표시: 논문 본문에서 참고문헌이 **[숫자]** (`[4]`, `[18]` 등) 형식으로 인용되었을 경우만 수집합니다.
- 동일한 참고문헌이 여러 번 인용된 경우 `"Context"` 배열에 해당 문장들을 포함합니다.
- `"Context"`의 개별 문장은 **300자 이하**로 유지해야 합니다.

## 출력 예시
```json
{{
    "4": {{
        "Title": "참고문헌 제목1",
        "Counter": 5,
        "Context": ["첫 번째 인용 문장", "두 번째 인용 문장", "세 번째 인용 문장", "네 번째 인용 문장", "다섯 번째 인용 문장"]
    }},
    "18": {{
        "Title": "참고문헌 제목2",
        "Counter": 2,
        "Context": ["첫 번째 인용 문장", "두 번째 인용 문장"]
    }}
}}
```

## References
{references}

## Essay
{essay}
"""
}

In [16]:
# 인용횟수를 카운팅하여 타겟 논문과 참고 논문 사이의 형식적 관련성을 평가합니다. 
n = 5
result_ = []
for index in range(n):
    result = []
    for key in reference_count_keys:
        for essay in output[key]:
            user_message = user_message_template_dict[str(index)].format(references=reference_dict, essay=essay)
            response = message_to_openai(user_message)
            try:
                text = response.choices[0].message.content
                text = re.sub("```json","",text)
                text = re.sub("```","",text)
                text_data = json.loads(text)
                result.append(text_data)
            except: 
                text_data = None
            # items['References'] = text_data
    result_.append(result)

    for data in result:
        for key, value_dict in data.items():
            output["References"][key]['Counter'] += value_dict['Counter']
            output["References"][key]['Context'].extend(value_dict['Context'])

In [18]:
import arxiv
import requests
from pathlib import Path

def fetch_arxiv_paper(title, max_results=30):
    
    search = arxiv.Search(
        query=title,
        max_results=max_results, 
        sort_by=arxiv.SortCriterion.Relevance
    )

    for result in search.results():
        if title[10:-10].lower().replace(" ", "") in result.title.lower().replace(" ", ""):
            return ( {
                "title": result.title,
                "abstract": result.summary,
                "pdf_url": result.pdf_url
            })
        
    return None 

def download_arxiv_pdf(pdf_url, save_path):
    response = requests.get(pdf_url, stream=True)
    if response.status_code == 200:
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
    
for index in range(len(output['References'])):
    title = output['References'][str(index+1)]['Title']
    try :
        paper_info = fetch_arxiv_paper(title)
        if paper_info is None:
            paper_info = fetch_arxiv_paper(title, 150)
    except Exception as e:
        print(index+1,"번째 논문 예외 발생: ", e)
        try :
            paper_info = fetch_arxiv_paper(title, None)
        except:
            paper_info = None

    if paper_info is not None:
        pdf_url = paper_info['pdf_url']
        abstract = paper_info['abstract']
        output['References'][str(index+1)]['abstract'] = abstract
        output['References'][str(index+1)]['pdf_url'] = pdf_url
        save_path = Path("references") / (str(index+1)+ "-" + paper_info['title'][:15]+".pdf")
        download_arxiv_pdf(pdf_url, save_path)
        print(index+1,"번째 논문 다운로드 완료")
    
    else:
        pdf_url = None
        abstract = None
        output['References'][str(index+1)]['abstract'] = abstract
        output['References'][str(index+1)]['pdf_url'] = pdf_url
        print(index+1,"번째 논문 다운로드 실패")
        print(f"    {output['References'][str(index+1)]['Title']}")

/tmp/ipykernel_950/372705134.py:13: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


1 번째 논문 다운로드 완료
2 번째 논문 다운로드 완료
3 번째 논문 다운로드 완료
4 번째 논문 다운로드 완료
5 번째 논문 다운로드 완료
6 번째 논문 다운로드 완료
7 번째 논문 다운로드 완료
8 번째 논문 다운로드 완료
9 번째 논문 다운로드 완료
10 번째 논문 다운로드 완료
11 번째 논문 다운로드 완료
12 번째 논문 다운로드 완료
13 번째 논문 다운로드 완료
14 번째 논문 다운로드 완료
15 번째 논문 다운로드 완료
16 번째 논문 다운로드 실패
    Measuring causal effects of data statistics on language model’s ’factual’ predictions
17 번째 논문 다운로드 완료
18 번째 논문 다운로드 완료
19 번째 논문 다운로드 완료
20 번째 논문 다운로드 완료
21 번째 논문 다운로드 실패
    Olmo: Accelerating the science of language models
22 번째 논문 다운로드 실패
    Investigating learning dynamics of bert fine-tuning
23 번째 논문 다운로드 실패
    Training compute-optimal large language models
24 번째 논문 다운로드 완료
25 번째 논문 다운로드 완료
26 번째 논문 다운로드 완료
27 번째 논문 다운로드 완료
28 번째 논문 다운로드 완료
29 번째 논문 다운로드 완료
30 번째 논문 다운로드 실패
    Starcoder: may the source be with you!
31 번째 논문 다운로드 완료
32 번째 논문 다운로드 완료
33 번째 논문 다운로드 완료
34 번째 논문 다운로드 완료
35 번째 논문 다운로드 완료
36 번째 논문 다운로드 완료
37 번째 논문 다운로드 완료
38 번째 논문 다운로드 완료
39 번째 논문 다운로드 완료
40 번째 논문 다운로드 완료
41 번째 논문 다운로드 완료
42 번째 논문 다운로드 실패
 

In [19]:
# 다운이 안받아진 논문은 제외합니다. 
# 해당 셀 위에 수동으로 넣으면 이를 반영시킬 수 있습니다.
filtered_reference_dict = { key: value for key, value in output['References'].items() if value['pdf_url'] is not None}


In [17]:
# # 다운로드 실패된 논문은 직접 가져오고, abstract도 직접 작성해서 넣어야 합니다.
# output['References']['16']['abstract'] = "Large amounts of training data are one of the major reasons for the high performance of state-of-the-art NLP models. But what exactly in the training data causes a model to make a certain prediction? We seek to answer this question by providing a language for describing how training data influences predictions, through a causal framework. Importantly, our framework bypasses the need to retrain expensive models and allows us to estimate causal effects based on observational data alone. Addressing the problem of extracting factual knowledge from pretrained language models (PLMs), we focus on simple data statistics such as co-occurrence counts and show that these statistics do influence the predictions of PLMs, suggesting that such models rely on shallow heuristics. Our causal framework and our results demonstrate the importance of studying datasets and the benefits of causality for understanding NLP models."
# output['References']['21']['abstract'] = "Language models (LMs) have become ubiquitous in both NLP research and in commercial product offerings. As their commercial importance has surged, the most powerful models have become closed off, gated behind proprietary interfaces, with important details of their training data, architectures, and development undisclosed. Given the importance of these details in scientifically studying these models, including their biases and potential risks, we believe it is essential for the research community to have access to powerful, truly open LMs. To this end, we have built OLMo, a competitive, truly Open Language Model, to enable the scientific study of language models. Unlike most prior efforts that have only released model weights and inference code, we release OLMo alongside open training data and training and evaluation code. We hope this release will empower the open research community and inspire a new wave of innovation."
# output['References']['22']['abstract'] = """
# The recently introduced pre-trained language
# model BERT advances the state-of-the-art on
# many NLP tasks through the fine-tuning approach, but few studies investigate how the
# fine-tuning process improves the model performance on downstream tasks. In this paper, we inspect the learning dynamics of BERT
# fine-tuning with two indicators. We use JS
# divergence to detect the change of the attention mode and use SVCCA distance to examine the change to the feature extraction mode
# during BERT fine-tuning. We conclude that
# BERT fine-tuning mainly changes the attention mode of the last layers and modifies the
# feature extraction mode of the intermediate
# and last layers. Moreover, we analyze the consistency of BERT fine-tuning between different random seeds and different datasets. In
# summary, we provide a distinctive understanding of the learning dynamics of BERT finetuning, which sheds some light on improving
# the fine-tuning results."""
# output['References']['23']['abstract'] = "We investigate the optimal model size and number of tokens for training a transformer language model under a given compute budget. We find that current large language models are significantly undertrained, a consequence of the recent focus on scaling language models whilst keeping the amount of training data constant. By training over 400 language models ranging from 70 million to over 16 billion parameters on 5 to 500 billion tokens, we find that for compute-optimal training, the model size and the number of training tokens should be scaled equally: for every doubling of model size the number of training tokens should also be doubled. We test this hypothesis by training a predicted compute-optimal model, Chinchilla, that uses the same compute budget as Gopher but with 70B parameters and 4× more more data. Chinchilla uniformly and significantly outperforms Gopher (280B), GPT-3 (175B), Jurassic-1 (178B), and Megatron-Turing NLG (530B) on a large range of downstream evaluation tasks. This also means that Chinchilla uses substantially less compute for fine-tuning and inference, greatly facilitating downstream usage. As a highlight, Chinchilla reaches a state-of-the-art average accuracy of 67.5% on the MMLU benchmark, greater than a 7% improvement over Gopher."
# output['References']['30']['abstract'] = "The BigCode community, an open-scientific collaboration working on the responsible development of Large Language Models for Code (Code LLMs), introduces StarCoder and StarCoderBase: 15.5B parameter models with 8K context length, infilling capabilities and fast large-batch inference enabled by multi-query attention. StarCoderBase is trained on 1 trillion tokens sourced from The Stack, a large collection of permissively licensed GitHub repositories with inspection tools and an opt-out process. We fine-tuned StarCoderBase on 35B Python tokens, resulting in the creation of StarCoder. We perform the most comprehensive evaluation of Code LLMs to date and show that StarCoderBase outperforms every open Code LLM that supports multiple programming languages and matches or outperforms the OpenAI code-cushman-001 model. Furthermore, StarCoder outperforms every model that is fine-tuned on Python, can be prompted to achieve 40\% pass@1 on HumanEval, and still retains its performance on other programming languages. We take several important steps towards a safe open-access model release, including an improved PII redaction pipeline and a novel attribution tracing tool, and make the StarCoder models publicly available under a more commercially viable version of the Open Responsible AI Model license."
# output['References']['44']['abstract'] = "State-of-the-art pre-trained language models have been shown to memorise facts and perform well with limited amounts of training data. To gain a better understanding of how these models learn, we study their generalisation and memorisation capabilities in noisy and low-resource scenarios. We find that the training of these models is almost unaffected by label noise and that it is possible to reach near-optimal results even on extremely noisy datasets. However, our experiments also show that they mainly learn from high-frequency patterns and largely fail when tested on low-resource tasks such as few-shot learning and rare entity recognition. To mitigate such limitations, we propose an extension based on prototypical networks that improves performance in low-resource named entity recognition tasks."
# output['References']['45']['abstract'] = """
# Large language models have achieved success
# on a number of downstream tasks, particularly
# in a few and zero-shot manner. As a consequence, researchers have been investigating
# both the kind of information these networks
# learn and how such information can be encoded
# in the parameters of the model. We survey
# the literature on changes in the network during
# training, drawing from work outside of NLP
# when necessary, and on learned representations
# of linguistic features in large language models.
# We note in particular the lack of sufficient research on the emergence of functional units –
# subsections of the network where related functions are grouped or organized – within large
# language models, and motivate future work that
# grounds the study of language models in an
# analysis of their changing internal structure during training time.
# """
# output['References']['48']['abstract'] = """
# We introduce LLaMA, a collection of foundation language models ranging from 7B to 65B parameters. We train our models on trillions of tokens, and show that it is possible to train state-of-the-art models using publicly available datasets exclusively, without resorting to proprietary and inaccessible datasets. In particular, LLaMA-13B outperforms GPT-3 (175B) on most benchmarks, and LLaMA-65B is competitive with the best models, Chinchilla-70B and PaLM-540B. We release all our models to the research
# """



In [20]:
user_message_template = """
다음 문장들을 분석하여 의미적으로 유사한 문장은 모두 제거하고, 서로 독립적인 문장만 남겨주세요.

### 입력 문장 목록:
{text_list}

### 출력 형식:
- 중복되거나 유사한 문장을 제거한 후, 의미적으로 독립적인 문장만 번호와 함께 남겨주세요.
- 출력 결과는 원래 문장의 의미를 유지하면서 중복을 피하도록 정제해주세요.

### 결과 예시 :
1. [27] reported that the performance of LLMs adheres to a scaling law, correlating positively with both the model size and the size of the pretraining corpus.
2. In Eq.1, the definition of the local acquisition maxima is also dependent on the injected knowledge k and the window size tw, but we write tLAM(q, i) for brevity.
3. While our finding that effectivity remains unchanged for different stages of pretraining may seem contradictory to the widely known observation that the amount of pretraining data is a critical factor in the performance of LLMs [23, 27], we suggest a plausible hypothesis based on further observations in §4.3.

위와 같은 방식으로 유사한 문장을 제거하고 정제된 결과를 반환해주세요.

"""
ratio = 0.2
nums = int(round(len(output['References']) * ratio, 0))
related_reference = dict(sorted(filtered_reference_dict.items(), key=lambda x:x[1]['Counter'], reverse=True)[:nums])

for index in related_reference.keys():
    query_list = related_reference[index]['Context']
    user_message = user_message_template.format(text_list=query_list)
    response = message_to_openai(user_message)
    related_reference[index]['Questions'] = response.choices[0].message.content

In [49]:
from tqdm import tqdm
user_message_template = """
다음 문서를 보고 질문은 한국어로 번역하여 출력해주세요.
질문에 대한 답변 및 관련 지식을 한국어로 요약하고, 답변의 근거가 되는 부분을 논문에서 그대로 발췌해 주세요.
그리고 모든 질문에 대한 답변을 500자 내외의 한 단락으로 정리해주세요.
### 인용 논문 제목 (Title): {title}
### 질문 목록 (Questions): {questions}
### 문서 (Document): {essay}
### 답변 예시 (Example Answer):
인용 논문 제목 (Title): <<논문 제목>>
1. **질문 :** <<질문1-한국어로 번역된 질문>>
   - **답변 :** <<질문1에 대한 한국어 답변이나 관련 지식 한 단락(300자 내외)>>
   - **근거 :** <<인용 논문에서 해당 답변을 뒷받침하는 문장>>

2. **질문 :** <<질문2-한국어로 번역된 질문>>
   - **답변 :** <<질문2에 대한 한국어 답변이나 관련 지식 한 단락(300자 내외)>>
   - **근거 :** <<인용 논문에서 해당 답변을 뒷받침하는 문장>>
...

답변 요약 
: <<전체 답변에 대한 요약 한 단락(500자 내외)>>
### 답변 요약 (Summary of Answers)
"""

user_message_template_2 = """
다음은 논문과 해당 논문의 참고문헌에 관한 내용입니다.
해당 논문이 인용 논문에서의 주요 질의 응답에 대하여 어떻게 연구를 발전시켰는지 한 단락으로 서술해주세요.

### 해당 논문 제목 (Title): {title}
### 해당 논문 내용 (Essay): {essay}
### 참고 문헌에 대한 주요 질의 응답(QnA): {qna}

### 답변 예시 (Example Answer):
인용 논문 제목 (Title): <<논문 제목>>
<<참고 문헌에 대한 주요 질의 응답에 대하여 어떻게 논문이 연구를 발전시켰는지 300자 내외로 서술>>
"""



data_dir = Path('references')
threshold = 25
main_essay = "\n\n".join([output[key] for key in ['Title','Authors','Abstract']])
for key in ['Introduction', 'Related Work', 'Experimental Setup', 'Results', 'Discussion and Conclusions']:
    for doc in output[key]:
        main_essay += (doc.page_content + "\n\n")

for index in tqdm(related_reference.keys(), desc="인용 논문과의 관련 지점 정리..."):
   title = related_reference[index]['Title']
   questions = related_reference[index]['Questions']
   
   # 논문 파일 찾기
   for path in data_dir.rglob("*.pdf"):
      if path.name.split("-")[0] == index:
         break
   
   # PDF 로드
   loader = UnstructuredPDFLoader(path)
   documents = loader.load()
   essay = documents[0].page_content
   essay = "\n".join([text for text in essay.split("\n") if len(text) >= threshold])
   user_message = user_message_template.format(essay = essay, questions=questions, title=title)
   response = message_to_openai(user_message)
   summary = response.choices[0].message.content
   related_reference[index]['Summary'] = summary
   
   user_message_2 = user_message_template_2.format(title=output['Title'], essay=main_essay, qna = summary)
   response = message_to_openai(user_message_2)
   summary_qna = response.choices[0].message.content
   related_reference[index]['Summary_QnA'] = summary_qna

인용 논문과의 관련 지점 정리...: 100%|██████████| 11/11 [04:45<00:00, 25.92s/it]


In [50]:
for index in related_reference.keys():
    print(related_reference[index]['Summary'])
    print()
    print(related_reference[index]['Summary_QnA'])
    print("####")

인용 논문 제목 (Title): Deduplicating training data makes language models better

1. **질문 :** LLMs는 훈련 데이터의 상당 부분을 기억한다. 모델의 크기가 커질수록 훈련 데이터를 기억하는 경향이 증가하며, 이는 지식의 일반화 능력에 해를 끼치지 않는다.
   - **답변 :** 대형 언어 모델(LLM)은 훈련 데이터의 상당 부분을 기억하는 경향이 있으며, 모델의 크기가 커질수록 이러한 경향이 더욱 두드러진다. 그러나 이 기억은 모델의 일반화 능력에 부정적인 영향을 미치지 않는다. 연구에 따르면, LLM은 훈련 데이터의 1% 이상을 그대로 출력할 수 있으며, 이는 모델의 크기와 관계없이 발생한다.
   - **근거 :** "Over 1% of tokens emitted unprompted from a model trained on standard datasets (e.g., C4) are part of a memorized sequence."

2. **질문 :** 우리는 LLM이 비인기 지식을 습득하는 데 어려움을 겪는다고 가정한다. 이는 충분한 노출이 필요하기 때문이다. 우리의 발견은 사전 훈련 코퍼스를 중복 제거하면 LLM 성능이 향상된다는 것을 시사한다.
   - **답변 :** LLM은 비인기 지식을 습득하는 데 어려움을 겪으며, 이는 충분한 노출이 필요하기 때문이다. 중복 제거를 통해 훈련 데이터의 품질을 높이면 LLM의 성능이 향상된다. 중복된 시퀀스에 더 높은 확률을 부여하는 것을 방지하고, 습득한 일반화를 더 오래 유지할 수 있도록 돕는다.
   - **근거 :** "Our findings suggest that deduplicating the pretraining corpus improves LLM performance by preventing the model from assigning a higher probability to duplicated sequences."

3. *

In [55]:
reference_summary = ""
for i, index in enumerate(related_reference.keys()):
    reference_summary += f"{i+1}번째\n"+ related_reference[index]['Summary'] + "\n\n" + related_reference[index]['Summary_QnA'] + "####"


In [58]:
print(output['Basic_summary'])

### 1. 기본 정보
1) 제목: How Do Large Language Models Acquire Factual Knowledge During Pretraining?
2) 저자: Hoyeon Chang, Jinho Park, Seonghyeon Ye, Sohee Yang, Youngkyung Seo, Du-Seong Chang, Minjoon Seo

### 2. 연구 목적
1) 문제의식: 대형 언어 모델의 사실적 지식 습득 메커니즘
2) 설명: 최근 대형 언어 모델(LLM)이 상당한 사실적 지식을 저장할 수 있다는 관찰에도 불구하고, 이들이 사전 훈련 중 사실적 지식을 어떻게 습득하는지에 대한 이해는 제한적이다. 본 연구는 LLM의 사실적 지식 습득 과정을 분석하여, 훈련 데이터의 양, 훈련 단계, 모델 크기, 배치 크기 등의 다양한 조건이 지식 습득에 미치는 영향을 조사한다. 이를 통해 LLM의 훈련 동역학을 이해하고, 지식 습득의 메커니즘을 규명하고자 한다.

### 3. 연구 방법
1) 실험 방법: 연구진은 LLM의 중간 사전 훈련 체크포인트를 사용하여, 이전에 접하지 않은 목표 지식을 주입하고, 다양한 조건에서 사실적 지식의 습득 과정을 모니터링하였다. 
2) 데이터: FICTIONAL KNOWLEDGE 데이터셋을 구성하여, 허구적이지만 현실적인 개체에 대한 설명을 포함한 문장을 주입하였다. 이 데이터셋은 LLM이 훈련 중에 접하지 않은 지식을 포함한다.
3) 모델 및 분석 방법: OLMo 모델을 사용하여, 주입된 지식에 대한 로그 확률을 평가하고, 지식 습득의 효과성 및 유지 가능성을 측정하기 위해 다양한 메트릭을 정의하였다.

### 4. 주요 결과
1) 연구의 주요 발견: LLM은 사실적 지식을 습득할 때, 미세한 확률의 누적을 통해 이루어지며, 훈련 중 지식이 주어지지 않으면 잊어버리는 경향이 있다. 또한, 모델 크기가 클수록 지식 습득의 효과성이 높아지지만, 훈련 데이터의 양이 많아져도 효과성은 크게 개선되지 않는다.
2) 기여 및 성과: